In [29]:
import scraping_class
import requests
import random
import tqdm
import pandas as pd
import time
import numpy as np
import datetime
import pytest
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile)

from multiprocessing import Queue, cpu_count
from threading import Thread
from numpy.random import randint
import logging

# similarity/distance measures
from scipy.spatial import distance
from sklearn.metrics.pairwise import linear_kernel

# for vectorization 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from selenium import webdriver
import re
import os
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

In [30]:
dirname = os.getcwd()
path = os.path.join(dirname, 'chromedriver')
driver = webdriver.Chrome(executable_path=path)

Problemer med en cookie pop-up

In [ ]:
# Finds and extracts all urls from a search term
search_term = 'a'
url = f'https://ekstrabladet.dk/find/?g=true&q={search_term}'
driver.get(url)
time.sleep(1)
# more comments button
next_page = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[4]/div[1]/div/div/div/div[32]/button')
# show all comment

loops = 0
url_pattern = 'https:\/\/[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}[-a-zA-Z0-9@:%_\+.~#?&//=;]*'
for i in range(100000): # how many times to press next
    loops += 1
    try:
        next_page.click() # when this shows an error, i.e. no more comments, it breaks.
        time.sleep(0.5)
        if loops %100 == 0: # save every 100 loops
            # get soup
            search_soup = BeautifulSoup(driver.page_source, 'lxml')
            articles = search_soup.find_all(class_='flex-item mar-l--b width-1of1') # links soup
            print(f'{loops} out of 100,000')
            urls = []
            
            for i in range(len(articles)):
                urls.append(re.findall(url_pattern, str(articles[i]))) # save url
            
            print(f'there are now {len(urls)} urls')
            
            # save to cvs
            df_urls = pd.DataFrame(data=urls)
            df_urls.to_csv("urls.csv", index=False)
    except: # if there are no more articles
        print('stop') # stop :I
        break

In [8]:
# save soup
with open("search_html.txt", "w", encoding="utf-8") as f:
    f.write(search_soup.get_text())
    f.close()

In [3]:
# load urls
urls = pd.read_csv("urls.csv")

In [4]:
# remove plus articles
urls_no_plus = [ url for url in urls['0'] if "https://ekstrabladet.dk/plus" not in url ]

In [94]:
i = 0
list1 = []
'''
Scrape headlines, number of comments, and time.
'''
def get_head(list, n):
    for url in urls_no_plus:
        global i
        no_comments = False
        data_list = []
        i +=1
        if i%100 == 0:
            print(f'{i} out of {len(urls_no_plus)}')
        if i == n: # break after n articles
            break

        data_list.append(url) # save url to df
        driver.get(url) # open the current url
        time.sleep(0.5)
        url_soup = BeautifulSoup(driver.page_source, 'lxml') # save soup


        # get ammount of comments
        while True:
            try:
                comments = url_soup.find_all(id='fnTalkCommentText') # div with the comment link
                comments = re.findall('\d+',comments[0].text)
                data_list.append(int(comments[0]))
                break
            except:
                data_list.append(np.nan)
                no_comments = True
                break

        # skips url if no comments
        if no_comments: 
            continue


        # get headline
        while True:
            try:
                headline = url_soup.find_all(class_='art-title') 
                headline = re.sub('\\n {16}','',headline[0].text)
                data_list.append(headline)
                break
            except:
                data_list.append(np.nan)
                break
        
        # get subtitle
        while True:
            try:
                subtitle = url_soup.find_all(class_='art-subtitle') 
                data_list.append(subtitle[0].text)
                break
            except:
                data_list.append(np.nan)
                break
        
        
        # get bread
        while True:
            try:
                bread = url_soup.find_all(class_='article-bodytext') 
                bread = re.sub(r'\n','',bread[0].text)
                bread = re.sub(r'      .*$','',bread)
                data_list.append(bread)
                break
            except:
                data_list.append(np.nan)
                break

        # get date
        while True:
            try:
                date = url_soup.find(class_='eb-row article-timestamp').get_text()
                date=re.findall("\d{1,2}. \w+. \d{4}",date)
                data_list.append(date[0])
                break
            except:
                data_list.append(np.nan)
                break

        # save to list of lists
        list1.append(data_list)
    return list1

In [36]:
# convert to df and drop nan
df = pd.DataFrame(list1).dropna()
df=df.reset_index(drop=True)

x=len(df)
dato=[]
for i  in range(x):
    data_time_string=df[3][i]
    x=re.sub('\.',"",data_time_string)
    y=datetime.datetime.strptime(x, '%d %b %Y')
    dato.append(y)
    time.sleep(0.5)
    

df['time']=pd.Series(dato).values

In [32]:
#split into equal chunks of urls.
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
chunks = list(chunks(urls_no_plus, 8800))
print(f'there are {len(chunks)} chunks')
chunk0 = chunks[0]
chunk1 = chunks[1]
chunk2 = chunks[2]
chunk3 = chunks[3]
# save chunks
chunk0 = pd.DataFrame(data=chunk0)
chunk0.to_csv("chunk0.csv", index=False)
chunk1 = pd.DataFrame(data=chunk1)
chunk1.to_csv("chunk1.csv", index=False)
chunk2 = pd.DataFrame(data=chunk2)
chunk2.to_csv("chunk2.csv", index=False)
chunk3 = pd.DataFrame(data=chunk3)
chunk3.to_csv("chunk3.csv", index=False)

there are 4 chunks


In [33]:
chunk0

,0
0,https://ekstrabladet.dk/flash/filmogtv/helt-ny...
1,https://ekstrabladet.dk/nyheder/erhvervsnyhede...
2,https://ekstrabladet.dk/sport/fodbold/landshol...
3,https://ekstrabladet.dk/nyheder/samfund/corona...
4,https://ekstrabladet.dk/ferie/tokyo-faar-genne...
...,...
8795,https://ekstrabladet.dk/biler/nu-er-det-slut-b...
8796,https://ekstrabladet.dk/sport/formel_1/transfe...
8797,https://ekstrabladet.dk/nyheder/samfund/dansk-...
8798,https://ekstrabladet.dk/flash/dkkendte/efter-n...


In [43]:
dupe_chunk = chunk1.append(chunk0).append(chunk2).append(chunk3).reset_index().duplicated(subset=None, keep='first')

In [46]:
dupe_chunk

0        False
1        False
2        False
3        False
4        False
         ...  
34964    False
34965    False
34966    False
34967    False
34968    False
Length: 34969, dtype: bool

In [47]:
for url in dupe_chunk:
    if url:
        print('Dupe!!!')

In [ ]:
'''
Saves the comments in a list.
Does not save arcticle headline etc.
'''
# open comments
comments= []
i = 0 # to only run loop x times
for i, url in enumerate(urls):
    # initiate check variables
    i2 = 0
    i3 = 0
    no_comments = False
    
    i +=1
    print(f"i = {i}")
    if i == 50: # break after 10 articles
        break
    
    print(url[0])
    driver.get(url[0]) # open the current url
    time.sleep(5)
    '''
    Tries to find the show more comments button.
    break after 10 seconds of trying
    '''
    while True: # infinite loop
        try:
            # show comments button
            show_comments = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[4]/div[3]/div[2]/div')
            show_comments.click() # click it
            time.sleep(5)
            break
        except:
            time.sleep(1) # waits a total of 10 seconds i2*10 for the page to load
            i2 +=1
            print(f"i2 = {i2}")
            if i2 == 5:
                no_comments = True
                break
   
    if no_comments: # next article
        continue
    '''
    Tries to find the comment url.
    if the url is blank wait and try again.
    break after 10 seconds
    '''
    while True: # infinite loop
        try:
            show_comments_soup = BeautifulSoup(driver.page_source, 'lxml') # save soup
            comment_link = show_comments_soup.find_all(id='talkStream_iframe') # div with the comment link
            comment_url = re.findall(url_pattern, str(comment_link)) # save the comment url
            print(comment_url)
            if comment_url == []:
                i3 += 1
                print(f"i3 = {i3}")
                time.sleep(1)
                if i3 == 5:
                    break
                continue
            break
        except:
            break
            
    # open the comment_url to save comments, break if not possible
    while True: # infinite loop
        try:
            driver.get(comment_url[0])
            time.sleep(1)
            break
        except:
            break
    
    '''
    tries to show all comments.
    when there are no more next_comments it breaks, also breaks if the button does not exist
    '''
    while True: # infinite loop
        try:
            # more comments button
            next_comment = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div/div/div[3]/div[2]/div/div/div/div/div[3]/button')
            next_comment.click()
        except:
            time.sleep(1)
            break
    
    '''
    save the extracted comments to list.
    breaks if no comments were extracted
    '''
    while True:
        try:
            comment_soup = BeautifulSoup(driver.page_source, 'lxml')
            comments_ = comment_soup.find_all(class_="talk-plugin-rich-text-text CommentContent__content___ZGv1q")
            if comments_ == []:
                break
            else:
                comments.append(comments_)
                break
        except:
            break

In [46]:
# save the comments:
df_comments = pd.DataFrame(data=comments)
df_comments.to_csv("comments.csv", index=False)

**Delete below me**

In [274]:
# CountVectorizer has a build-in tokenizer and lowercases by default. Also has an option to remove stopwords.
vectorizer = CountVectorizer()
# However, you can override the default tokenization with your own defined function, like so:
#vectorizer = CountVectorizer(tokenizer=preprocess)

# fit and transform train
df_bow = vectorizer.fit_transform(df[1].values)
# Only tranform test: never fitting your vectorizer on the test set (it is cheating). OOV words are handled automatically be sklearn's vectorizer.
#X_test_bow = vectorizer.transform(df_test.review.values)

In [276]:
tfidf = TfidfVectorizer()
df_tfidf = tfidf.fit_transform(df[1].values)

In [288]:
# https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
cosine_similarities = linear_kernel(df_tfidf[3], df_tfidf).flatten()

indices = cosine_similarities.argsort()[::-1] # in descending order 
print("most similar:",indices[:10])
print("least similar", indices[-9:])

most similar: [ 3 51 33 74 45 26 21 22 23 24]
least similar [46 54 47 48 49 50 52 53  0]


In [289]:
print(df[1].values[3])
print()
print("most similar: ", df[1].values[51])
print()
print("least similar: ", df[1].values[46])

Officielt: Tager over i FC Barcelona

most similar:  Barcelonas præsident melder Ronald Koeman tæt på FC Barcelona

least similar:  Fængsel skal tage imod coronasmittet
